### loading dataset

In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 1000)
data = pd.read_csv(r'../2. data_cleaning\4.1 some cols FEed.csv')
data.head(3)

,property_name,sector,society,area_in_sqft,price_in_crores,price_per_sqft,areaWithType,bedRoom,bathroom,balcony,additionalRoom,address,floorNum,facing,agePossession,nearbyLocations,description,furnishDetails,features,rating,property_type,noOfFloor,area
0,2 BHK Flat in Sector 70 Gurgaon,sector 70,shree vardhman victoria,1300.0,1.10,8461.0,Carpet area: 1300 (120.77 sq.m.),2.0,2.0,2,not available,"Sector 70 Gurgaon, Gurgaon, Haryana",8.0,north-east,1 to 5 Year Old,"['Airia Mall', 'Golf Course Extn Road', 'Sohna Road', 'Southern Peripheral Rd, Gurugram', 'Kunskapsskolan International', 'Ektaa Hospitals Main Sohna Rd', 'Sanjeevani Hospital - Child Specialist', 'Federal Bank Sector 71', 'Central Bank Of India Sohna Rd', 'Axis Bank, Sohna Rd', 'Spaze Palazo, Golf Course Ext Rd', 'VATIKA BUSINESS PARK Sohna Rd', 'The Medicity, Spaze iTech Park', 'Tulip Violet Society, Sector 69', 'IndianOil, Hasanpur']","Square yards rera reg.\r\nHrera 660/2017/307\r\nProject rera reg.\r\n70 of 2017\r\nShree vardhman victoria - Project info\r\nShree vardhman victoria is an exclusive project located in sector 70, southern peripheral road and well connected by major road(s) like sohna road, southern peripheral road. The total area in which shree vardhman victoria has been built is 11 acre. This project has been developed by shree vardhman who are one of the reputed developers in the gurgaon. The project current status is ready to move. It has 592 units. The apartment units are available in various configurations like 2 bhk flats at a price of rs 82.44 l with property size of 1300 sqft, 3 bhk flats at a price of rs 1.24 cr with property size of 1950 sqft, 4 bhk flats at a price of rs 1.62 cr with property size of 2550 sqft.\r\nShree vardhman victoria - Price list\r\nUnit type area new home price*\r\n2 bhk 1300 sq. Ft.\r\nApartment \r\n1300 sq. Ft.\r\n(Saleable)\r\n 82.44 lac\r\n3 bhk 1950 sq. Ft.\r\nA...","['3 Wardrobe', '1 Modular Kitchen', '1 Chimney', 'No AC', 'No Bed', 'No Curtains', 'No Dining Table', 'No Exhaust Fan', 'No Fan', 'No Geyser', 'No Light', 'No Microwave', 'No Fridge', 'No Sofa', 'No Stove', 'No TV', 'No Washing Machine', 'No Water Purifier']","['Security / Fire Alarm', 'Power Back-up', 'Feng Shui / Vaastu Compliant', 'Intercom Facility', 'Lift(s)', 'High Ceiling Height', 'Maintenance Staff', 'False Ceiling Lighting', 'Water Storage', 'Separate entry for servant room', 'No open drainage around', 'Piped-gas', 'Swimming Pool', 'Park', 'Internet/wi-fi connectivity', 'Shopping Centre', 'Fitness Centre / GYM', 'Waste Disposal', 'Rain Water Harvesting', 'Club house / Community Center', 'Water softening plant']","['Green Area5 out of 5', 'Construction5 out of 5', 'Management4.5 out of 5', 'Amenities4.5 out of 5', 'Connectivity3.5 out of 5']",flat,1.0,NaN
1,3 Bedroom House for sale in Sector 40 Gurgaon,sector 40,NaN,900.0,2.15,23889.0,Plot area 100(83.61 sq.m.),3.0,3.0,3,"Pooja Room,Study Room,Store Room","Sector 40 Gurgaon, Gurgaon, Haryana",0.0,NaN,5 to 10 year old,"['Huda city centre metro station', 'Axis bank ATM', 'State bank of india ATM', 'Icici bank ATM', 'Icici ATM', 'Citi bank ATM', 'Axis bank ATM', 'Hdfc ATM', 'Standard chartered ATM', 'Hdfc bank ATM', 'Axis bank ATM', 'Hdfc bank ATM', 'State bank of india ATM', 'Dispencery', 'Shivam Hospital Gurgaon', 'Fortis Memorial Research Institute Fortis Vivekanand Hospital', 'Dr. Naval Mendiratta', 'Centre For Sight Gurgaon Sector 29', 'Sukhmani Hospital Pvt. Ltd', 'Ahmed Hospital Multi Speciality', ""DR AKRAM JAWED'S THE UPPER LIMB CLINIC"", 'Medanta', 'Gardian Pharmacy', 'City Medical', 'Gardian Pharmacy', 'Pernod Ricard Charitable Dispensary', 'IBP Petrol Pump', 'Bharat petroleum', 'Hdfc bank', 'Raj Restaurant', 'Fast food', 'Fast Food', 'Om Sweets', 'Bar and restaurant', 'Cafe Coffee Day', '32nd Milestone', 'Darbar', 'PWO house', 'Dhabba', 'St. Angels Jr', 'School of Inspired Leadership SOIL', 'CR Model Public School', 'Manav Rachna School', 'St. Angels Sr', 'Amity Global School', 'Salvan

### FEing `areaWithType` col

In [3]:
data['areaWithType'].head(3)

0    Carpet area: 1300 (120.77 sq.m.)
1          Plot area 100(83.61 sq.m.)
2         Plot area 520(434.79 sq.m.)
Name: areaWithType, dtype: object

#### `area_in_sqft` col contains one of the 3 area_types in `areaWithType` col.

In [4]:
data[['area_in_sqft', 'areaWithType']].sample(3)

,area_in_sqft,areaWithType
2190,773.0,Built Up area: 761 (70.7 sq.m.)
2203,1935.0,Plot area 215(179.77 sq.m.)
2531,1094.0,Super Built up area 1360(126.35 sq.m.)Carpet area: 1094 sq.ft. (101.64 sq.m.)


#### creating `superBuiltupArea` col

In [5]:
# areaWithType containing "super" keyword
data[data['areaWithType'].str.lower().str.strip().fillna('').str.contains('super')]['areaWithType']

3                                                                                       Super Built up area 2010(186.74 sq.m.)
11                                                                                      Super Built up area 2217(205.97 sq.m.)
15                                                                                        Super Built up area 700(65.03 sq.m.)
16      Super Built up area 1424(132.29 sq.m.)Built Up area: 1200 sq.ft. (111.48 sq.m.)Carpet area: 1139 sq.ft. (105.82 sq.m.)
17                                                                                      Super Built up area 1960(182.09 sq.m.)
                                                                 ...                                                          
4068                                             Super Built up area 1990(184.88 sq.m.)Carpet area: 1700 sq.ft. (157.94 sq.m.)
4071                                                                                    Super Built up area 195

In [6]:
data['SuperBuiltupArea'] = data['areaWithType'].str.lower().str.strip().str.extract(r'super built up area (\d+\.?\d+)')
data['SuperBuiltupArea']

0        NaN
1        NaN
2        NaN
3       2010
4        NaN
        ... 
4077     NaN
4078    1780
4079     NaN
4080     NaN
4081     NaN
Name: SuperBuiltupArea, Length: 4082, dtype: object

In [7]:
data['SuperBuiltupArea'].value_counts(dropna=False)[data['SuperBuiltupArea'].value_counts(dropna=False) >20]

SuperBuiltupArea
NaN     2148
1650      38
1950      38
2000      26
1578      25
2150      23
1640      22
Name: count, dtype: int64

In [8]:
# observation
# 2148 places regex couldn't extract superBuiltupArea value.

In [9]:
data[data['SuperBuiltupArea'].isna()][['SuperBuiltupArea', 'areaWithType']].sample(5)

,SuperBuiltupArea,areaWithType
1703,NaN,Carpet area: 830 (77.11 sq.m.)
638,NaN,Plot area 320(267.56 sq.m.)
924,NaN,Carpet area: 1760 (163.51 sq.m.)
2978,NaN,Built Up area: 1081 (100.43 sq.m.)
608,NaN,Carpet area: 1295 (120.31 sq.m.)


#### creating `BuiltupArea` col

In [10]:
data['areaWithType'].str.lower().str.strip().sample(3)

3412    built up area: 3000 (278.71 sq.m.)
1097    built up area: 1210 (112.41 sq.m.)
3646           plot area 163(136.29 sq.m.)
Name: areaWithType, dtype: object

In [11]:
data['BuiltupArea'] = data['areaWithType'].str.lower().str.strip().str.extract(r'built up area\s*:\s*(\d+\.?\d*)')
data['BuiltupArea']

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
       ... 
4077    NaN
4078    NaN
4079    NaN
4080    NaN
4081    NaN
Name: BuiltupArea, Length: 4082, dtype: object

In [12]:
data['BuiltupArea'].value_counts(dropna=False)[data['BuiltupArea'].value_counts(dropna=False) >20]

BuiltupArea
NaN     2848
1900      34
1600      26
2000      25
1300      24
1700      22
Name: count, dtype: int64

In [13]:
# observation
# 2848 places regex couldn't extract BuiltupArea value.

In [14]:
data[data['BuiltupArea'].isna()][['BuiltupArea', 'areaWithType']].sample(5)

,BuiltupArea,areaWithType
3016,NaN,Super Built up area 1711(158.96 sq.m.)
2500,NaN,Carpet area: 1439 (133.69 sq.m.)
3404,NaN,Plot area 500(46.45 sq.m.)
2209,NaN,Super Built up area 2087(193.89 sq.m.)
2948,NaN,Carpet area: 1500 (139.35 sq.m.)


#### creating `CarpetArea` col

In [15]:
data['areaWithType'].str.lower().str.strip().sample(3)

4042         carpet area: 366.08 (34.01 sq.m.)
913           carpet area: 1150 (106.84 sq.m.)
2666    super built up area 3400(315.87 sq.m.)
Name: areaWithType, dtype: object

In [16]:
data['CarpetArea'] = data['areaWithType'].str.lower().str.strip().str.extract(r'carpet area\s*:\s*(\d+\.?\d*)')
data['CarpetArea']

0       1300
1        NaN
2        NaN
3        NaN
4       3500
        ... 
4077     NaN
4078     NaN
4079    2295
4080    1120
4081    1300
Name: CarpetArea, Length: 4082, dtype: object

In [17]:
data['CarpetArea'].value_counts(dropna=False)[data['CarpetArea'].value_counts(dropna=False) >20]

CarpetArea
NaN     2079
1400      42
1800      37
1600      36
1200      33
1500      30
1350      29
1650      28
900       25
2000      23
1300      23
1000      23
1450      23
2200      21
Name: count, dtype: int64

In [18]:
# observation
# 2079 places regex couldn't extract CarpetArea value.

In [19]:
data[data['CarpetArea'].isna()][['CarpetArea', 'areaWithType']].sample(5)

,CarpetArea,areaWithType
1436,NaN,Super Built up area 1200(111.48 sq.m.)
877,NaN,Super Built up area 1997(185.53 sq.m.)
2081,NaN,Super Built up area 1755(163.04 sq.m.)
2610,NaN,Super Built up area 2812(261.24 sq.m.)
604,NaN,Super Built up area 1550(144 sq.m.)


#### converting all 3 area_cols from sqm to sqft

In [20]:
import re
def convert_to_sqft(area_text, area_value):
    if not area_value:
        return None
    match = re.search(r"{} \((\d+\.?\d*) sq.m.\)*".format(area_value), str(area_text))
    if match:
        sqm_value = float(match.group(1))
        return sqm_value*10.76391
    return area_value

In [21]:
data['CarpetArea'] = data.apply(lambda x: convert_to_sqft(x['areaWithType'], x['CarpetArea']), axis=1)
data['BuiltupArea'] = data.apply(lambda x: convert_to_sqft(x['areaWithType'], x['BuiltupArea']), axis=1)
data['SuperBuiltupArea'] = data.apply(lambda x: convert_to_sqft(x['areaWithType'], x['SuperBuiltupArea']), axis=1)

In [22]:
data[['SuperBuiltupArea', 'BuiltupArea',	'CarpetArea']].head(3)

,SuperBuiltupArea,BuiltupArea,CarpetArea
0,NaN,NaN,1299.957411
1,NaN,NaN,NaN
2,NaN,NaN,NaN


#### checking where none of the 3 area values are present

In [37]:
data[(data['CarpetArea'].isnull() & data['BuiltupArea'].isnull() & data['SuperBuiltupArea'].isnull())][['CarpetArea', 'BuiltupArea', 'SuperBuiltupArea']].shape[0]

735

In [33]:
data.shape[0]

4082

In [38]:
data[(data['CarpetArea'].isnull() & data['BuiltupArea'].isnull() & data['SuperBuiltupArea'].isnull())][['CarpetArea', 'BuiltupArea', 'SuperBuiltupArea']].sample(5)

,CarpetArea,BuiltupArea,SuperBuiltupArea
2440,NaN,NaN,NaN
2515,NaN,NaN,NaN
1533,NaN,NaN,NaN
499,NaN,NaN,NaN
3113,NaN,NaN,NaN


In [39]:
# there are 737 rows with no are at all, it is becoz some rows also have plot_area which we are yet to extract

#### extracting plot_area values from areaWithType col

In [40]:
data['areaWithType']

0             Carpet area: 1300 (120.77 sq.m.)
1                   Plot area 100(83.61 sq.m.)
2                  Plot area 520(434.79 sq.m.)
3       Super Built up area 2010(186.74 sq.m.)
4             Carpet area: 3500 (325.16 sq.m.)
                         ...                  
4077               Plot area 300(250.84 sq.m.)
4078    Super Built up area 1780(165.37 sq.m.)
4079          Carpet area: 2295 (213.21 sq.m.)
4080          Carpet area: 1120 (104.05 sq.m.)
4081          Carpet area: 1300 (120.77 sq.m.)
Name: areaWithType, Length: 4082, dtype: object